In [1]:
import nzpy
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
import time

In [2]:
#Netezza Cloud Connection Information
nz_host             = "150.239.60.252"
nz_port             = 5480
nz_user             = "admin"
nz_password         = "OWU5MzkwMzhjYzI3YTFmNmEyZmQ0OTY1"

nz_histdb           = "HISTDBI"  
nz_schema           = "ADMIN"
nzqh_table          = "NZ_QUERY_HISTORY"
nzsu_table          = "NZ_SYSUTIL_HISTORY"

nzqh_tsubmit_start  = ""
nzqh_tsubmit_end    = ""
nzqh_where          = ""
nzqh_and            = ""

In [3]:
nzcon = nzpy.connect(user=nz_user, password=nz_password, host=nz_host, database=nz_histdb, port=nz_port)

bool(nzcon)

True

In [4]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [5]:

sql_start = "SELECT ENTRY FROM {}.{}".format(nz_schema, nzsu_table)
entry_df = pd.read_sql_query(sql_start, nzcon)
entry_df.columns = entry_df.columns.str.decode('utf-8')

# Pull just the date in format YYYY-MM-DD 
entry_df['DATE_MEAN'] = pd.to_datetime(entry_df['ENTRY']).dt.date


In [6]:
dropdown_date = widgets.Dropdown(options = ['MONTH_MEAN', 'WEEKDAY_MEAN', 'HOUR_MEAN', 'DATE_MEAN', 'DATE_AND_HOUR_MEAN', 'MONTH_AND_YEAR_MEAN', 'YEAR_MEAN', 'DAY_MEAN'], description='Aggregation Index:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
dropdown_date_start = widgets.Dropdown(options = unique_sorted_values_plus_ALL(entry_df.DATE_MEAN), description='Start Date:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
dropdown_date_end = widgets.Dropdown(options = unique_sorted_values_plus_ALL(entry_df.DATE_MEAN), description='End Date:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
button = widgets.Button(description='Click to Graph')
out = widgets.Output()
output_date = widgets.Output()
plot_output = widgets.Output()

def common_filtering(date_start, date_end, type_date):
    output_date.clear_output()
    plot_output.clear_output()


    if (date_start == ALL) & (date_end == ALL):
        starting_date = entry_df['DATE_MEAN'].min()
        ending_date = entry_df['DATE_MEAN'].max()
    elif (date_start == ALL):
        starting_date = entry_df['DATE_MEAN'].min()
        ending_date = date_end
    elif (date_end == ALL):
        starting_date = date_start
        ending_date = entry_df['DATE_MEAN'].max()
    else:
        starting_date = date_start
        ending_date = date_end
        
    nzsu_submit_start  = "{} 00:00:00".format(starting_date)
    nzsu_submit_end    = "{} 23:59:59".format(ending_date)
    nzsu_where          = "WHERE ENTRY >= '{}' and ENTRY <= '{}'".format(nzsu_submit_start,nzsu_submit_end)
           
    sql_dates = "SELECT ENTRY, SPU_CPU, SPU_DISK, SPU_FABRIC, SPU_MEMORY FROM {}.{} {}".format(nz_schema, nzsu_table, nzsu_where)
    df1 = pd.read_sql_query(sql_dates, nzcon)
    df1.columns = df1.columns.str.decode('utf-8')
    
    if (type_date == 'MONTH_MEAN'):
        df1['MONTH_MEAN'] = pd.to_datetime(df1['ENTRY']).dt.month  # AVERAGE_MONTH
        
    if (type_date == 'WEEKDAY_MEAN'):
        # Function to Convert Date in Format MM-DD-YYY to day of the Week
        weekDays = ("0 Monday", "1 Tuesday", "2 Wednesday", "3 Thursday", "4 Friday", "5 Saturday", "6 Sunday")
    
        def convert_to_weekday(date):
            dt = datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
            return weekDays[dt.weekday()]
        
        date_list = df1['ENTRY'].tolist()
        new_column = []
        for i in date_list:
            new_column.append(convert_to_weekday(i))

        df1['WEEKDAY_MEAN'] = new_column
        
    if (type_date == 'HOUR_MEAN'):
        df1['HOUR_MEAN'] = pd.to_datetime(df1['ENTRY']).dt.strftime("%H")
     
    if (type_date == 'DATE_MEAN'):
        df1['DATE_MEAN'] = pd.to_datetime(df1['ENTRY']).dt.date  # ENTRY_DATE

    if (type_date == 'DATE_AND_HOUR_MEAN'):
        df1['HOUR_MEAN'] = pd.to_datetime(df1['ENTRY']).dt.strftime("%H")
        df1['DATE_MEAN'] = pd.to_datetime(df1['ENTRY']).dt.date  # ENTRY_DATE
        df1['DATE_AND_HOUR_MEAN'] = df1['DATE_MEAN'].astype(str) + "-" + df1['HOUR_MEAN'].astype(str)  #day by hour

    if (type_date == 'MONTH_AND_YEAR_MEAN'):
        df1['YEAR_MEAN'] = pd.to_datetime(df1['ENTRY']).dt.year
        df1['MONTH_MEAN'] = pd.to_datetime(df1['ENTRY']).dt.month  # AVERAGE_MONTH
        df1['MONTH_AND_YEAR_MEAN'] = df1['YEAR_MEAN'].astype(str) + "-" + df1['MONTH_MEAN'].astype(str) #YEAR_MONTH
        
    if (type_date == 'YEAR_MEAN'):
        df1['YEAR_MEAN'] = pd.to_datetime(df1['ENTRY']).dt.year
        
    if (type_date == 'DAY_MEAN'):
        df1['DAY_MEAN'] = pd.to_datetime(df1['ENTRY']).dt.strftime("%d")     # ENTRY_DAY
        
    SPU_CPU_DF = df1.groupby([type_date])['SPU_CPU'].mean()
    SPU_DISK_DF = df1.groupby([type_date])['SPU_DISK'].mean()
    SPU_FABRIC_DF = df1.groupby([type_date])['SPU_FABRIC'].mean()
    SPU_MEMORY_DF = df1.groupby([type_date])['SPU_MEMORY'].mean()

    SPU_CPU_DataFrame = pd.DataFrame(SPU_CPU_DF)
    SPU_DISK_DataFrame = pd.DataFrame(SPU_DISK_DF)
    SPU_FABRIC_DataFrame = pd.DataFrame(SPU_FABRIC_DF)
    SPU_MEMORY_DataFrame = pd.DataFrame(SPU_MEMORY_DF)
    
    CPU_DISK_MERG = pd.merge(SPU_CPU_DataFrame, SPU_DISK_DataFrame, on=type_date)
    FABRIC_MEMORY_MERG = pd.merge(SPU_FABRIC_DataFrame, SPU_MEMORY_DataFrame, on=type_date)

    merged_df1 = pd.merge(CPU_DISK_MERG, FABRIC_MEMORY_MERG, on=type_date)
    merged_df1[type_date] = merged_df1.index
    common_filter = merged_df1
    
    with output_date:
            display(common_filter)
    
    with plot_output:
        plt.plot(merged_df1[type_date], merged_df1['SPU_CPU'], label = "SPU_CPU")
        plt.plot(merged_df1[type_date], merged_df1['SPU_DISK'], label = "SPU_DISK")
        plt.plot(merged_df1[type_date], merged_df1['SPU_FABRIC'], label = "SPU_FABRIC")
        plt.plot(merged_df1[type_date], merged_df1['SPU_MEMORY'], label = "SPU_MEMORY")

        plt.xlabel(type_date, labelpad=20)
        plt.ylabel('Average USAGE')
        
        if (type_date == 'DATE_AND_HOUR_MEAN' and len(merged_df1[type_date]) > 48):
            plt.xticks(merged_df1[type_date][::10])
        else:
            plt.xticks(merged_df1[type_date])
        
        plt.xticks(rotation = 90)
        plt.grid()
        plt.legend(loc='upper center', bbox_to_anchor=(1.5, 1.05),
            ncol=3, fancybox=True, shadow=True)
        plt.title('Average Usage by ' + type_date + ' between ' + str(starting_date) + ' and ' + str(ending_date))
        plt.rcParams["figure.figsize"] = (30,30)
        plt.rcParams.update({'font.size': 45})
        plt.show()
        
    
def on_button_clicked(_):
      with out:
        out.clear_output()
        common_filtering(dropdown_date_start.value, dropdown_date_end.value, dropdown_date.value)

button.on_click(on_button_clicked)

# Usage over Time

In [7]:
display(dropdown_date_start)
display(dropdown_date_end)
display(dropdown_date)
display(button)
display(output_date)
display(plot_output)

Dropdown(description='Start Date:', layout=Layout(width='max-content'), options=('ALL', datetime.date(2020, 10…

Dropdown(description='End Date:', layout=Layout(width='max-content'), options=('ALL', datetime.date(2020, 10, …

Dropdown(description='Aggregation Index:', layout=Layout(width='max-content'), options=('MONTH_MEAN', 'WEEKDAY…

Button(description='Click to Graph', style=ButtonStyle())

Output()

Output()

In [8]:
sql_start = "SELECT QH_TSUBMIT FROM {}.{} LIMIT 50000".format(nz_schema, nzqh_table)

start_time = time.time()
entry_df2 = pd.read_sql_query(sql_start, nzcon)
end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

entry_df2.columns = entry_df2.columns.str.decode('utf-8')

# Pull just the date in format YYYY-MM-DD 
entry_df2['DATE_MEAN'] = pd.to_datetime(entry_df2['QH_TSUBMIT']).dt.date


## takes about 3 minutes for 3 million entries

sql execute time:2.6143343448638916


In [9]:
dropdown_date2 = widgets.Dropdown(options = ['MONTH_MEAN', 'WEEKDAY_MEAN', 'HOUR_MEAN', 'DATE_MEAN', 'DATE_AND_HOUR_MEAN', 'MONTH_AND_YEAR_MEAN', 'YEAR_MEAN', 'DAY_MEAN'], description='Aggregation Index:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
dropdown_date_start2 = widgets.Dropdown(options = unique_sorted_values_plus_ALL(entry_df2.DATE_MEAN), description='Start Date:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
dropdown_date_end2 = widgets.Dropdown(options = unique_sorted_values_plus_ALL(entry_df2.DATE_MEAN), description='End Date:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
button2 = widgets.Button(description='Click to Graph')
out2 = widgets.Output()
output_date2 = widgets.Output()
plot_output2 = widgets.Output()


def common_filtering2(date_start2, date_end2, type_date2):
    output_date2.clear_output()
    plot_output2.clear_output()
    
    if (date_start2 == ALL) & (date_end2 == ALL):
        starting_date2 = entry_df2['DATE_MEAN'].min()
        ending_date2 = entry_df2['DATE_MEAN'].max()
    elif (date_start2 == ALL):
        starting_date2 = entry_df2['DATE_MEAN'].min()
        ending_date2 = date_end2
    elif (date_end2 == ALL):
        starting_date2 = date_start2
        ending_date2 = entry_df2['DATE_MEAN'].max()
    else:
        starting_date2 = date_start2
        ending_date2 = date_end2

    nzqh_submit_start  = "{} 00:00:00".format(starting_date2)
    nzqh_submit_end    = "{} 23:59:59".format(ending_date2)
    nzqh_where          = "WHERE QH_TSUBMIT >= '{}' and QH_TSUBMIT <= '{}'".format(nzqh_submit_start,nzqh_submit_end)

    start_time = time.time()
    sql_dates = "SELECT QH_USER, QH_DATABASE, QH_SQL, QH_TSUBMIT FROM {}.{} {} LIMIT 1000".format(nz_schema, nzqh_table, nzqh_where)
    df2 = pd.read_sql_query(sql_dates, nzcon)
    df2.columns = df2.columns.str.decode('utf-8')
    
    if (type_date2 == 'MONTH_MEAN'):
        df2['MONTH_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.month  # AVERAGE_MONTH
        df2['YEAR_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.year
        df2['MONTH_AND_YEAR_MEAN'] = df2['YEAR_MEAN'].astype(str) + "-" + df2['MONTH_MEAN'].astype(str) #YEAR_MONTH
        
        df22 = df2.groupby(['MONTH_AND_YEAR_MEAN']).agg({'QH_SQL': 'count', 'MONTH_MEAN': 'first'})
        
    if (type_date2 == 'WEEKDAY_MEAN'):
        # Function to Convert Date in Format MM-DD-YYY to day of the Week
        weekDays = ("0 Monday", "1 Tuesday", "2 Wednesday", "3 Thursday", "4 Friday", "5 Saturday", "6 Sunday")
    
        def convert_to_weekday(date):
            dt = datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
            return weekDays[dt.weekday()]
        
        date_list = df2['QH_TSUBMIT'].tolist()
        new_column = []
        for i in date_list:
            new_column.append(convert_to_weekday(i))

        df2['WEEKDAY_MEAN'] = new_column
        df2['DATE_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.date  # ENTRY_DATE
        
        df22 = df2.groupby(['DATE_MEAN']).agg({'QH_SQL': 'count', 'WEEKDAY_MEAN': 'first'})
        
    if (type_date2 == 'HOUR_MEAN'):
        df2['HOUR_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.strftime("%H")
        df2['DATE_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.date  # ENTRY_DATE
        df2['DATE_AND_HOUR_MEAN'] = df2['DATE_MEAN'].astype(str) + "-" + df2['HOUR_MEAN'].astype(str)  #day by hour
       
        df22 = df2.groupby(['DATE_AND_HOUR_MEAN']).agg({'QH_SQL': 'count', 'HOUR_MEAN': 'first'})
        
    if (type_date2 == 'DATE_MEAN'):
        df2['DATE_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.date  # ENTRY_DATE
        df22 = df2.groupby(['DATE_MEAN']).agg({'QH_SQL': 'count', 'DATE_MEAN': 'first'})

    if (type_date2 == 'DATE_AND_HOUR_MEAN'):
        df2['HOUR_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.strftime("%H")
        df2['DATE_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.date  # ENTRY_DATE
        df2['DATE_AND_HOUR_MEAN'] = df2['DATE_MEAN'].astype(str) + "-" + df2['HOUR_MEAN'].astype(str)  #day by hour
        df22 = df2.groupby(['DATE_AND_HOUR_MEAN']).agg({'QH_SQL': 'count', 'DATE_AND_HOUR_MEAN': 'first'})


    if (type_date2 == 'MONTH_AND_YEAR_MEAN'):
        df2['YEAR_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.year
        df2['MONTH_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.month  # AVERAGE_MONTH
        df2['MONTH_AND_YEAR_MEAN'] = df2['YEAR_MEAN'].astype(str) + "-" + df2['MONTH_MEAN'].astype(str) #YEAR_MONTH
        df22 = df2.groupby(['MONTH_AND_YEAR_MEAN']).agg({'QH_SQL': 'count', 'MONTH_AND_YEAR_MEAN': 'first'})

    if (type_date2 == 'YEAR_MEAN'):
        df2['YEAR_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.year
        df22 = df2.groupby(['YEAR_MEAN']).agg({'QH_SQL': 'count', 'YEAR_MEAN': 'first'})

    if (type_date2 == 'DAY_MEAN'):
        df2['DAY_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.strftime("%d")     # ENTRY_DAY
        df2['DATE_MEAN'] = pd.to_datetime(df2['QH_TSUBMIT']).dt.date  # ENTRY_DATE
        df22 = df2.groupby(['DATE_MEAN']).agg({'QH_SQL': 'count', 'DAY_MEAN': 'first'})
    
    df22.index.names = ['Date']
        
    MEAN_DF = df22.groupby([type_date2]).mean()
    SUM_DF = df22.groupby([type_date2]).sum()

    merged_df2 = pd.merge(MEAN_DF, SUM_DF, on=type_date2)
    merged_df2.columns = ['AVG', 'COUNT']
    merged_df2.reset_index(inplace=True)
    merged_df2 = merged_df2.rename(columns = {'index':'Date'})

    
    common_filter2 = merged_df2
    
    with output_date2:
            display(common_filter2)
    
    with plot_output2:
        merged_df2.plot(x=type_date2, y=["AVG", "COUNT"], kind="bar")
        plt.xlabel(type_date2, labelpad = 20)
        plt.ylabel('QUERY COUNT')
        plt.xticks(rotation = 90)
        plt.grid()
        plt.legend(loc='upper center', bbox_to_anchor=(1.5, 1.05),
            ncol=3, fancybox=True, shadow=True)
        plt.title('QUERY VOLUME by  ' + type_date2 + ' between ' + str(starting_date2) + ' and ' + str(ending_date2))
        plt.rcParams["figure.figsize"] = (20,30)
        plt.rcParams.update({'font.size': 22})
        plt.show()
        end_time = time.time()
        print("sql execute time:%s" % (end_time - start_time))
    
            


def on_button_clicked2(_):
      with out2:
        out2.clear_output()
        common_filtering2(dropdown_date_start2.value, dropdown_date_end2.value, dropdown_date2.value)

button2.on_click(on_button_clicked2)


# Volume Over Time

In [11]:
display(dropdown_date_start2)
display(dropdown_date_end2)
display(dropdown_date2)
display(button2)
display(output_date2)
display(plot_output2)

Dropdown(description='Start Date:', layout=Layout(width='max-content'), options=('ALL', datetime.date(2020, 10…

Dropdown(description='End Date:', layout=Layout(width='max-content'), options=('ALL', datetime.date(2020, 10, …

Dropdown(description='Aggregation Index:', layout=Layout(width='max-content'), options=('MONTH_MEAN', 'WEEKDAY…

Button(description='Click to Graph', style=ButtonStyle())

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '   MONTH_MEAN   AVG  COUNT\n0         …

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 432x288 with 1 Axes>', 'i…

In [12]:
dropdown_date3 = widgets.Dropdown(options = ['MONTH_MEAN', 'WEEKDAY_MEAN', 'HOUR_MEAN', 'DATE_MEAN', 'DATE_AND_HOUR_MEAN', 'MONTH_AND_YEAR_MEAN', 'YEAR_MEAN', 'DAY_MEAN'], description='Aggregation Index:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
dropdown_date_start3 = widgets.Dropdown(options = unique_sorted_values_plus_ALL(entry_df2.DATE_MEAN), description='Start Date:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
dropdown_date_end3 = widgets.Dropdown(options = unique_sorted_values_plus_ALL(entry_df2.DATE_MEAN), description='End Date:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
button3 = widgets.Button(description='Click to Graph')
out3 = widgets.Output()
output_date3 = widgets.Output()
plot_output3 = widgets.Output()


def common_filtering3(date_start3, date_end3, type_date3):
    output_date3.clear_output()
    plot_output3.clear_output()
    
    if (date_start3 == ALL) & (date_end3 == ALL):
        starting_date3 = entry_df2['DATE_MEAN'].min()
        ending_date3 = entry_df2['DATE_MEAN'].max()
    elif (date_start3 == ALL):
        starting_date3 = entry_df2['DATE_MEAN'].min()
        ending_date3 = date_end3
    elif (date_end3 == ALL):
        starting_date3 = date_start3
        ending_date3 = entry_df2['DATE_MEAN'].max()
    else:
        starting_date3 = date_start3
        ending_date3 = date_end3

    nzqh_submit_start  = "{} 00:00:00".format(starting_date3)
    nzqh_submit_end    = "{} 23:59:59".format(ending_date3)
    nzqh_where          = "WHERE QH_TSUBMIT >= '{}' and QH_TSUBMIT <= '{}'".format(nzqh_submit_start,nzqh_submit_end)

    start_time = time.time()
    sql_dates = "SELECT QH_TSUBMIT, QH_TSTART, QH_TEND FROM {}.{} {} LIMIT 1000".format(nz_schema, nzqh_table, nzqh_where)
    df3 = pd.read_sql_query(sql_dates, nzcon)
    df3.columns = df3.columns.str.decode('utf-8')
    
    df3['Q_TIME'] = (pd.to_datetime(df3['QH_TSTART']) - pd.to_datetime(df3['QH_TSUBMIT'])).dt.total_seconds()
    df3['RUNTIME'] = (pd.to_datetime(df3['QH_TEND']) - pd.to_datetime(df3['QH_TSTART'])).dt.total_seconds()

    
    if (type_date3 == 'MONTH_MEAN'):
        df3['MONTH_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.month  # AVERAGE_MONTH
        df3['YEAR_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.year
        
    if (type_date3 == 'WEEKDAY_MEAN'):
        # Function to Convert Date in Format MM-DD-YYY to day of the Week
        weekDays = ("0 Monday", "1 Tuesday", "2 Wednesday", "3 Thursday", "4 Friday", "5 Saturday", "6 Sunday")
    
        def convert_to_weekday(date):
            dt = datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
            return weekDays[dt.weekday()]
        
        date_list = df3['QH_TSUBMIT'].tolist()
        new_column = []
        for i in date_list:
            new_column.append(convert_to_weekday(i))

        df3['WEEKDAY_MEAN'] = new_column
        
    if (type_date3 == 'HOUR_MEAN'):
        df3['HOUR_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.strftime("%H")
        
    if (type_date3 == 'DATE_MEAN'):
        df3['DATE_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.date  # ENTRY_DATE
       
    if (type_date3 == 'DATE_AND_HOUR_MEAN'):
        df3['HOUR_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.strftime("%H")
        df3['DATE_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.date  # ENTRY_DATE
        df3['DATE_AND_HOUR_MEAN'] = df3['DATE_MEAN'].astype(str) + "-" + df3['HOUR_MEAN'].astype(str)  #day by hour
        

    if (type_date3 == 'MONTH_AND_YEAR_MEAN'):
        df3['YEAR_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.year
        df3['MONTH_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.month  # AVERAGE_MONTH
        df3['MONTH_AND_YEAR_MEAN'] = df3['YEAR_MEAN'].astype(str) + "-" + df3['MONTH_MEAN'].astype(str) #YEAR_MONTH
        
    if (type_date3 == 'YEAR_MEAN'):
        df3['YEAR_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.year
        
    if (type_date3 == 'DAY_MEAN'):
        df3['DAY_MEAN'] = pd.to_datetime(df3['QH_TSUBMIT']).dt.strftime("%d")     # ENTRY_DAY
       
    
    QUEUE_TIME_DF = df3.groupby([type_date3])['Q_TIME'].sum()
    RUNTIME_DF = df3.groupby([type_date3])['RUNTIME'].sum()
        
    QUEUE_TIME_DataFrame = pd.DataFrame(QUEUE_TIME_DF)
    RUNTIME_DataFrame = pd.DataFrame(RUNTIME_DF)

    merged_df3 = pd.merge(QUEUE_TIME_DataFrame, RUNTIME_DataFrame, on=type_date3)

    merged_df3[type_date3] = merged_df3.index
        
    common_filter3 = merged_df3
    
    with output_date3:
            display(common_filter3)
    
    with plot_output3:
        plt.plot(merged_df3[type_date3], merged_df3['Q_TIME'], label = "Queue Time")
        plt.plot(merged_df3[type_date3], merged_df3['RUNTIME'], label = "Run Time")
       
    
        plt.xlabel(type_date3)
        if (type_date3 == 'DATE_AND_HOUR_MEAN' and len(merged_df3[type_date3]) > 48):
            plt.rc('font', size=30)
            plt.xticks(merged_df3[type_date3][::10])
        else:
            plt.rc('font', size=30)
            plt.xticks(merged_df3[type_date3])
            
        plt.ylabel('Time in Seconds')
        #plt.xticks(merged_df3[type_date3])
        plt.xticks(rotation = 75)
        plt.grid()
        plt.legend(loc='upper center', bbox_to_anchor=(1.5, 1.05),
            ncol=3, fancybox=True, shadow=True)
        plt.title('Runtime and Queue by  ' + type_date3 + ' between ' + str(starting_date3) + ' and ' + str(ending_date3))
        plt.rcParams["figure.figsize"] = (45,50)
        plt.rc('font', size=30)
        #plt.gcf()
        plt.show()
        end_time = time.time()
        print("sql execute time:%s" % (end_time - start_time))
    
            

def on_button_clicked3(_):
      with out3:
        out3.clear_output()
        common_filtering3(dropdown_date_start3.value, dropdown_date_end3.value, dropdown_date3.value)

button3.on_click(on_button_clicked3)


# Runtime and Queue Time over Time

In [13]:
display(dropdown_date_start3)
display(dropdown_date_end3)
display(dropdown_date3)
display(button3)
display(output_date3)
display(plot_output3)


Dropdown(description='Start Date:', layout=Layout(width='max-content'), options=('ALL', datetime.date(2020, 10…

Dropdown(description='End Date:', layout=Layout(width='max-content'), options=('ALL', datetime.date(2020, 10, …

Dropdown(description='Aggregation Index:', layout=Layout(width='max-content'), options=('MONTH_MEAN', 'WEEKDAY…

Button(description='Click to Graph', style=ButtonStyle())

Output()

Output()

In [16]:
dropdown_date4 = widgets.Dropdown(options = ['MONTH_MEAN', 'WEEKDAY_MEAN', 'HOUR_MEAN', 'DATE_MEAN', 'DATE_AND_HOUR_MEAN', 'MONTH_AND_YEAR_MEAN', 'YEAR_MEAN', 'DAY_MEAN'], description='Aggregation Index:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
dropdown_date_start4 = widgets.Dropdown(options = unique_sorted_values_plus_ALL(entry_df2.DATE_MEAN), description='Start Date:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
dropdown_date_end4 = widgets.Dropdown(options = unique_sorted_values_plus_ALL(entry_df2.DATE_MEAN), description='End Date:', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
dropdown_demographic = widgets.Dropdown(options = ['Users', 'Database'], description = 'Group By: ', layout = {'width': 'max-content'}, style={'description_width': 'initial'})
limit_slider = widgets.IntSlider(min=1, max=100, description = "Amount of Users/Databases: ", style={'description_width': 'max-content'}, layout=widgets.Layout(width='50%'))
button4 = widgets.Button(description='Click to Graph')
out4 = widgets.Output()
output_date4 = widgets.Output()
plot_output4 = widgets.Output()


def common_filtering4(date_start4, date_end4, type_date4, demo, limit_value):
    output_date4.clear_output()
    plot_output4.clear_output()
    
    if (date_start4 == ALL) & (date_end4 == ALL):
        starting_date4 = entry_df2['DATE_MEAN'].min()
        ending_date4 = entry_df2['DATE_MEAN'].max()
    elif (date_start4 == ALL):
        starting_date4 = entry_df2['DATE_MEAN'].min()
        ending_date4 = date_end4
    elif (date_end4 == ALL):
        starting_date4 = date_start4
        ending_date4 = entry_df2['DATE_MEAN'].max()
    else:
        starting_date4 = date_start4
        ending_date4 = date_end4

    nzqh_submit_start  = "{} 00:00:00".format(starting_date4)
    nzqh_submit_end    = "{} 23:59:59".format(ending_date4)
    nzqh_where          = "QH_TSUBMIT >= '{}' and QH_TSUBMIT <= '{}'".format(nzqh_submit_start,nzqh_submit_end)
    
    if (demo == 'Users'):
        nzqh_demo = "QH_USER"
    else:
        nzqh_demo = "QH_DATABASE"
        
    sql_find_top_demo = "SELECT {}, COUNT (QH_SQL) FROM {}.{} GROUP BY {} ORDER BY COUNT DESC LIMIT {}".format(nzqh_demo, nz_schema, nzqh_table, nzqh_demo, limit_value)
    limit_df = pd.read_sql_query(sql_find_top_demo, nzcon)
    limit_df.columns = limit_df.columns.str.decode('utf-8')
    demo_list = "("
    for x in limit_df[nzqh_demo]:
        demo_list = demo_list + "'" + x + "'" + ","

    demo_list = demo_list[:-1]
    demo_list = demo_list + ")"

    start_time = time.time()  
    sql_dates = "SELECT QH_TSUBMIT, QH_SQL, {} FROM {}.{} WHERE {} and {} in {} LIMIT 50000".format(nzqh_demo, nz_schema, nzqh_table, nzqh_where, nzqh_demo, demo_list)
    df4 = pd.read_sql_query(sql_dates, nzcon)
    df4.columns = df4.columns.str.decode('utf-8')
     
    
 
    if (type_date4 == 'MONTH_MEAN'):
        df4['MONTH_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.month  # AVERAGE_MONTH
        df4['YEAR_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.year
        
        
    if (type_date4 == 'WEEKDAY_MEAN'):
        # Function to Convert Date in Format MM-DD-YYY to day of the Week
        weekDays = ("0 Monday", "1 Tuesday", "2 Wednesday", "3 Thursday", "4 Friday", "5 Saturday", "6 Sunday")
    
        def convert_to_weekday(date):
            dt = datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')
            return weekDays[dt.weekday()]
        
        date_list = df4['QH_TSUBMIT'].tolist()
        new_column = []
        for i in date_list:
            new_column.append(convert_to_weekday(i))

        df4['WEEKDAY_MEAN'] = new_column
        
    if (type_date4 == 'HOUR_MEAN'):
        df4['HOUR_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.strftime("%H")
        
    if (type_date4 == 'DATE_MEAN'):
        df4['DATE_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.date  # ENTRY_DATE
       
    if (type_date4 == 'DATE_AND_HOUR_MEAN'):
        df4['HOUR_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.strftime("%H")
        df4['DATE_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.date  # ENTRY_DATE
        df4['DATE_AND_HOUR_MEAN'] = df4['DATE_MEAN'].astype(str) + "-" + df4['HOUR_MEAN'].astype(str)  #day by hour
        

    if (type_date4 == 'MONTH_AND_YEAR_MEAN'):
        df4['YEAR_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.year
        df4['MONTH_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.month  # AVERAGE_MONTH
        df4['MONTH_AND_YEAR_MEAN'] = df4['YEAR_MEAN'].astype(str) + "-" + df4['MONTH_MEAN'].astype(str) #YEAR_MONTH
        
    if (type_date4 == 'YEAR_MEAN'):
        df4['YEAR_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.year
        
    if (type_date4 == 'DAY_MEAN'):
        df4['DAY_MEAN'] = pd.to_datetime(df4['QH_TSUBMIT']).dt.strftime("%d")     # ENTRY_DAY
       
 
    DEMO_DF = df4.groupby([nzqh_demo, type_date4]).agg({'QH_SQL': 'count'})
    plt.rcParams.update({'font.size': 40})
    DEMO_DF.groupby([type_date4,nzqh_demo]).sum().unstack().plot(figsize=(30,15), kind='bar',y='QH_SQL', stacked=True, colormap="gist_rainbow")
    current_handles, _ = plt.gca().get_legend_handles_labels()
    reversed_handles = reversed(current_handles)
    labels = reversed(DEMO_DF.index.unique())
    plt.legend(reversed_handles,labels,loc='best')
    
    common_filter4 = DEMO_DF
    
    with output_date4:
            display(common_filter4)
    
    with plot_output4:
        plt.xlabel(type_date4)
        plt.ylabel('QUERY COUNT by ' + demo)
        
        plt.xticks(rotation = 90)
        plt.grid()
        plt.legend(loc='best',bbox_to_anchor=(1, 1),
            ncol=3, fancybox=True, shadow=True) #bbox_to_anchor=(1.5, 1.05)
        plt.title('QUERY VOLUME by  ' + type_date4 + ' between ' + str(starting_date4) + ' and ' + str(ending_date4))
        plt.show()
        end_time = time.time()
        print("sql execute time:%s" % (end_time - start_time))
            


def on_button_clicked4(_):
      with out4:
        out4.clear_output()
        common_filtering4(dropdown_date_start4.value, dropdown_date_end4.value, dropdown_date4.value, dropdown_demographic.value, limit_slider.value)

button4.on_click(on_button_clicked4)



# User and Database Volume Over Time

In [17]:
display(dropdown_date_start4)
display(dropdown_date_end4)
display(dropdown_date4)
display(dropdown_demographic)
display(limit_slider)
display(button4)
#display(output_date4)
display(plot_output4)


Dropdown(description='Start Date:', layout=Layout(width='max-content'), options=('ALL', datetime.date(2020, 10…

Dropdown(description='End Date:', layout=Layout(width='max-content'), options=('ALL', datetime.date(2020, 10, …

Dropdown(description='Aggregation Index:', layout=Layout(width='max-content'), options=('MONTH_MEAN', 'WEEKDAY…

Dropdown(description='Group By: ', layout=Layout(width='max-content'), options=('Users', 'Database'), style=De…

IntSlider(value=1, description='Amount of Users/Databases: ', layout=Layout(width='50%'), min=1, style=SliderS…

Button(description='Click to Graph', style=ButtonStyle())

Output()